In [16]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

df = pd.read_csv('fer2013.csv')
df = df[df['label'] != 'disgust']

feature_columns = [col for col in df.columns if col not in ['image_id', 'label', 'split']]
X = df[feature_columns]
y = df['label']

print(f"Number of features: {len(feature_columns)}")
print(f"Target classes: {sorted(y.unique())}")

train_mask = df['split'] == 'train'
test_mask = df['split'] == 'test'

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Scale the features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Number of features: 52
Target classes: ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Training set size: 25233
Test set size: 6321


In [17]:
# Train SVC model
print("Training SVC model...")
svc_model = SVC(
    kernel='rbf',  # Radial basis function kernel
    C=1.0,         # Regularization parameter
    gamma='scale', # Kernel coefficient
    random_state=42
)

# Train the model
svc_model.fit(X_train_scaled, y_train)
print("Model training completed!")


Training SVC model...
Model training completed!


In [ ]:
# Make predictions and evaluate
# Make predictions
y_train_pred = svc_model.predict(X_train_scaled)
y_test_pred = svc_model.predict(X_test_scaled)

# Calculate accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Default
# Training Accuracy: 0.6270
# Test Accuracy: 0.5667

# No Disgust
# Training Accuracy: 0.6347
# Test Accuracy: 0.5740

Training Accuracy: 0.6347
Test Accuracy: 0.5740


In [19]:
# Detailed evaluation
print("Classification Report (Test Set):")
print(classification_report(y_test, y_test_pred))

print("\nConfusion Matrix (Test Set):")
cm = confusion_matrix(y_test, y_test_pred)
print(cm)


Classification Report (Test Set):
              precision    recall  f1-score   support

       angry       0.49      0.40      0.44       815
        fear       0.46      0.20      0.28       885
       happy       0.77      0.85      0.81      1683
     neutral       0.44      0.62      0.51      1162
         sad       0.44      0.43      0.43      1020
    surprise       0.68      0.70      0.69       756

    accuracy                           0.57      6321
   macro avg       0.55      0.53      0.53      6321
weighted avg       0.57      0.57      0.56      6321


Confusion Matrix (Test Set):
[[ 327   50   74  191  129   44]
 [ 110  176   97  224  171  107]
 [  48   15 1434  113   40   33]
 [  65   41  100  724  193   39]
 [  87   53   92  330  437   21]
 [  31   49   54   68   24  530]]


In [ ]:
# Optional - Hyperparameter tuning with GridSearch
from sklearn.model_selection import GridSearchCV

# Define parameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'linear']
}

print("Starting hyperparameter tuning...")
grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=3,  # 3-fold cross-validation
    scoring='accuracy',
    n_jobs=-1,  # Use all available cores
    verbose=1
)

# This might take a while depending on your data size
grid_search.fit(X_train_scaled, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")


In [ ]:
# Train final model with best parameters
best_svc = grid_search.best_estimator_
final_predictions = best_svc.predict(X_test_scaled)
final_accuracy = accuracy_score(y_test, final_predictions)

print(f"Final model test accuracy: {final_accuracy:.4f}")
print("\nFinal Classification Report:")
print(classification_report(y_test, final_predictions))
